# Scripted Primo Search Assessment Tool

Description: < Describe the purpose of the SPSAT>

## Load required libraries

In [1]:
import pandas as pd
import os
import time
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
import io
os.chdir('/Volumes/jwa_drive1/git/spst/ScriptedPrimoSearchAssessmentTool/dev')
from spst import SPST,CompResults

In [2]:
test = True
type(test)
test = str(test)
type(test)

str

## Define Some Variables

In [2]:
## files with search strings
f = '/Volumes/jwa_drive1/git/spst/search_strings.csv'
#f = '/Volumes/jwa_drive1/git/spst/cr_records.xml'
p = SPST()
s = SPST()
# search strings are shared by each instance of the class
p.set_search_strings(f,'title_author')
# set parameters sets the number of results per search, the 'onCampus' variable, and the 'institution' variable
# these variables are shared by each instance of the class
p.set_params('2','true','BOSU')

## the platform and scope are assigned by the set_platform_scope() function which requires a dict as the parameter.
## the platform, scope, and/or url_base can be assigned per instance. If the url_base is not passed, it is adjusted
## based on the default value for the platform
p_dict = {'platform':'production',
          'scope':'&loc=local,scope:(ALMA_BOSU1)&loc=adaptor,primo_central_multiple_fe&onCampus=true&json=true'}
s_dict  = {'platform':'stage',
           'scope':'&loc=local,scope:(ALMA_BOSU1)&loc=adaptor,primo_central_multiple_fe&onCampus=true&json=true',
          'url_base':'http://bu-primostage.hosted.exlibrisgroup.com:1701/PrimoWebServices/xservice/search/brief'}
s.set_platform_scope(s_dict)
p.set_platform_scope(p_dict)

In [3]:
search1 = p.data(start=0,end=10)
search1 = search1.sort_values(by=['Search_No','ResultNumber'], ascending=[1,1])
search2 = s.data(start=0,end=10)
search2 = search2.sort_values(by=['Search_No','ResultNumber'], ascending=[1,1])

Processed:  0  searches on  production
Processed:  10  searches on  production
Processed:  0  searches on  stage
Processed:  10  searches on  stage


In [4]:
comp = CompResults()
r_df = comp.Compare2ResultSets(search1,search2,columns = ['title','creator','recordid','fulltext'])

In [5]:
r_df.head(10)

pTitle  \
Search                   ResultNumber                                                      
murder                   1                                                        Murder   
                         2                                                 Felony murder   
mindfulness chronic pain 1             Acceptance and Commitment Therapy and Mindfuln...   
                         2             Mindfulness-oriented interventions for trauma ...   
gay marriage             1                                       Legalizing gay marriage   
                         2             Odd Couples A History of Gay Marriage in Scand...   
solar energy industry    1             DOE's role in the solar energy industry, and p...   
                         2             The structure of the solar energy industry hea...   
imigration policy        1                     What counts as racist immigration policy?   
                         2             Emigration laws and policies of Mongolia commu...   

                                                                                  sTitle  \
Search                   ResultNumber                                                      
murder                   1                                                        Murder   
                         2                                                 Felony murder   
mindfulness chronic pain 1             Acceptance and Commitment Therapy and Mindfuln...   
                         2             Mindfulness-oriented interventions for trauma ...   
gay marriage             1                                       Legalizing gay marriage   
                         2             Odd Couples A History of Gay Marriage in Scand...   
solar energy industry    1             Organizational Ingenuity and the Paradox of Em...   
                         2             Licensing arrangements and the development of ...   
imigration policy        1                     What counts as racist immigration policy?   
                         2             Emigration laws and policies of the Republic o...   

                                                                                pCreator  \
Search                   ResultNumber                                                      
murder                   1                                                           NaN   
                         2                                               Binder, Guyora.   
mindfulness chronic pain 1                        Mccracken, Lance M. ; Vowles, Kevin E.   
                         2                                                           NaN   
gay marriage             1                                               Mello, Michael.   
                         2                                              Rydström, Jens.   
solar energy industry    1             United States. Congress. Senate. Committee on ...   
                         2             United States. Congress. Senate. Select Commit...   
imigration policy        1                                                  Fox, Cybelle   
                         2                United States. President (1993-2001 : Clinton)   

                                                                                sCreator  \
Search                   ResultNumber                                                      
murder                   1                                                           NaN   
                         2                                               Binder, Guyora.   
mindfulness chronic pain 1                        Mccracken, Lance M. ; Vowles, Kevin E.   
                         2                                                           NaN   
gay marriage             1                                               Mello, Michael.   
                         2                                              Rydström, Jens.   
solar energy industry    1       

In [ ]:
# Define the aggregation calculations
aggregations = {
    'match' : {
        'match' : 'count'
    },
    'sType' : {
        'stype' : 'count'
    },
    'pType' : {
        'pType' : 'count'
        
    },

    'sToplevel' : {
        'pToplevel' : 'count'
        
    },
    'pToplevel' : {
        'pToplevel': 'count'
        
    }
}

In [ ]:
search1.columns
#cr_df = cr_df[['title', 'collection', 'creator', 'type','fulltext','delcategory','doi','OpenURL']]

In [ ]:
cr_df.columns

In [ ]:
alma_id = ''
counter = 0
doi_cnt = 0
for i,r in cr_df.iterrows():
    length = len(i[0]) -1
    #print(i)
    #print('Match: ', i[0][:-1].lower() == r['title'][:length].lower())
    #print(i[0][:-1].lower())
    #print(r['title'][:length].lower())
    #print(r['collection'])
    if 'ALMA_BOSU' in i[1]:
        alma_id = i[1]
        author = r['creator']
        title = r['title'].lower()
        counter += 1
    if (alma_id > '' and 'ALMA_BOSU' not in i[1]):
        if i[0][:-1].lower() == r['title'][:length].lower():
            doi_cnt += 1
            print(str(doi_cnt),' of ',str(counter))
            print(alma_id)
            print(author)
            print(title)
            print('Search: ',i[0][:-1].lower())
            print('Title:  ',r['title'][:length].lower())
            print('Creator:  ',r['creator'][:length].lower())
            print('Collection: ',r['collection'])
            print('Fulltext: ',r['fulltext'])
            try:
                if r['collection'] != 'PsycARTICLES (American Psychological Association)':
                    print(r['doi'].replace('.org/','.org.ezproxy.bu.edu/').replace('https','http'))
                print(r['OpenURL'])
            except Exception as e:
                print(r['OpenURL'])
        
            print()

In [ ]:
d = {}
c = 0
for k,group in by_search:
    print()
    print(k)
    print(group.columns)
    group = group[['pFulltext', 'pToplevel','pType','sType']]
    c+= 1
    print(group.groupby('pType').count())
    

Code below has been migrated to a class

In [ ]:
# build_url 
    # takes base_url and search_string, returns url properly formated for searching
def build_url(base_url,search_string):
    url = base_url+query_Params1 + search_string.replace(' ','+') + query_Params2 + query_Params3
    #print(url)
    return(url)
    
# parse_response 
    # takes response (as json) from search, returns response_dict (dict of our responses)
def parse_response(search,response,platform,search_no):
    response_dict = {}
    # define the elements to grab from the json response. These become the columns in the dataframe
    elements = []
    #elements.append('Platform')
    #elements.append('Search')
    #elements.append('ResultNumber')
    #elements.append('TotalHits')
    #elements.append('Rank')
    #elements.append('SearchEngine')
    elements.append('control.recordid')
    elements.append('display.type')
    elements.append('display.creator')
    elements.append('display.title')
    elements.append('display.ispartof')
    elements.append('delivery.delcategory')
    elements.append('delivery.fulltext')
    elements.append('search.searchscope')
    elements.append('search.general')
    elements.append('facets.collection')
    elements.append('facets.frbrtype')
    elements.append('facets.toplevel')
    elements.append('facets.prefilter')
    elements.append('sort')
    # parse the response to return the data desired elements
    response = json.loads(response.decode('utf8'))
    # need to extract the desired elements and add them to a dict
    response = response['SEGMENTS']
    response = response['JAGROOT']
    response = response['RESULT']
    response_dict['facet'] = response['FACETLIST']
    response_dict['docset'] = response['DOCSET']
    docs = response_dict['docset']['DOC']
    facets = response_dict['facet']
        
    ## define a dataframe to return the results
    doc_columns = ['Platform','Search_No','Search','ResultNumber','TotalHits','FirstHit','LastHit','Rank','SearchEngine','recordid', \
           'type','creator','title','ispartof','delcategory','fulltext','searchscope','general','collection', \
          'frbrtype','toplevel','prefilter','sort']
    df_doc = pd.DataFrame(columns = doc_columns)
    fac_columns = ['Platform','Search_No','Search','TotalHits','SearchEngine','facet','values']
    df_facets = pd.DataFrame(columns = fac_columns)
    #bib = doc['PrimoNMBib']['record']
    #print(type(bib)
    ## iterate through the results (normally 10) to populate a dictionary that will be added to the dataframe
    for doc in docs:
        doc_dict = {}
        doc_dict['Platform'] = platform
        doc_dict['Search_No'] = search_no
        doc_dict['Search'] = search
        doc_dict['ResultNumber'] = doc['@NO']
        doc_dict['TotalHits'] = response['DOCSET']['@TOTALHITS']
        doc_dict['FirstHit'] = response['DOCSET']['@FIRSTHIT']
        doc_dict['LastHit'] = response['DOCSET']['@LASTHIT']
        doc_dict['Rank'] = doc['@RANK']
        doc_dict['SearchEngine'] = doc['@SEARCH_ENGINE']
        #doc_dict['Bib'] = doc['PrimoNMBib']['record']
        Bib = doc['PrimoNMBib']['record']
        
        for element in elements:
            x = element.split('.')
            if len(x) == 1:
                try:
                    doc_dict[x[-1]] = Bib[x[0]]
                except:
                    pass
            if len(x) == 2:
                try:
                    doc_dict[x[-1]] = Bib[x[0]][x[1]]
                except:
                    pass
            if len(x) == 3:
                try:
                    doc_dict[x[-1]] = Bib[x[0]][x[1]][x[2]]
                except:
                    pass
            if len(x) == 4:
                try:
                    doc_dict[x[-1]] = Bib[x[0]][x[1]][x[2]][x[3]]
                except:
                    pass
            if len(x) == 5:
                try:
                    doc_dict[x[-1]] = Bib[x[0]][x[1]][x[2]][x[3]][x[4]]
                except:
                    pass   
        df_doc = df_doc.append(doc_dict, ignore_index=True)
        #print(doc_dict)
        f_dict = parse_facets(facets['FACET'])
        for k,v in f_dict.items() :
            if doc['@NO'] == '1':
                facet_dict = {}
                facet_dict['Platform'] = platform
                facet_dict['Search_No'] = search_no
                facet_dict['Search'] = search
                #facet_dict['ResultNumber'] = doc['@NO']
                facet_dict['TotalHits'] = response['DOCSET']['@TOTALHITS']
                facet_dict['SearchEngine'] = doc['@SEARCH_ENGINE']
                facet_dict['facet'] = k
                facet_dict['values'] = v
                df_facets = df_facets.append(facet_dict,ignore_index=True)

    return(df_doc,Bib,df_facets)


def parse_facets(facets):
    return_dict = {}
    for facet in facets:
        f = facet['@NAME']
        fv = facet['FACET_VALUES']
        #print(f)
        l = []
        for x in fv :
            try:
                l.append((x['@KEY'],int(x['@VALUE'])))
            except Exception as e:
                pass
        sorted_by_second = sorted(l, key=lambda tup: tup[1],reverse=True)
        return_dict[f] = sorted_by_second
    return return_dict

In [ ]:
## here we are going to combine values from the search on production and stage to compare the results for each search
columns = ['Search','match','pRecordid','sRecordid','pRank','sRank','pCollection','sCollection', 
                        'pCreator','sCreator', 'pDelcategory','sDelcategory','pFrbrtype','sFrbrtype', 
                        'pFulltext','sFulltext', 'pTitle','sTitle', 'pToplevel','sToplevel', 'pType','sType']
#comp_results = pd.DataFrame(columns = columns)
mask = search1['Platform'] == 'production'
production = search1[mask] 
mask = search1['Platform'] == 'stage'
stage = search1[mask]
stage= stage[['Rank', 'ResultNumber', 'Search', 'SearchEngine',
       'Search_No', 'TotalHits', 'collection', 'creator', 'delcategory',
       'frbrtype', 'fulltext', 'general', 'ispartof', 'prefilter', 'recordid',
       'searchscope', 'sort', 'title', 'toplevel', 'type']]
production = production[['Rank', 'ResultNumber', 'Search', 'SearchEngine',
       'Search_No', 'TotalHits', 'collection', 'creator', 'delcategory',
       'frbrtype', 'fulltext', 'general', 'ispartof', 'prefilter', 'recordid',
       'searchscope', 'sort', 'title', 'toplevel', 'type','doi','OpenURL']]
stage= stage.set_index(['Search_No', 'ResultNumber'])
#cr_df = production[['pRecordid', 'ResultNumber','Search', 'pTitle', 'pCollection',  'pType','pFulltext','pDelcategory']]
cr_df = production.set_index(['Search', 'recordid','ResultNumber'])
production = production.set_index(['Search_No', 'ResultNumber'])
production = production[['recordid','Rank', 'Search','collection', 'creator','delcategory', 'frbrtype', 'fulltext','title', 'toplevel','type']]
stage = stage[['recordid','Rank', 'Search','collection', 'creator','delcategory', 'frbrtype', 'fulltext','title', 'toplevel','type']]
production.columns = ['pRecordid','pRank', 'Search', 'pCollection', 'pCreator', 'pDelcategory', 'pFrbrtype',
       'pFulltext', 'pTitle', 'pToplevel', 'pType']
stage.columns = ['sRecordid','sRank', 'Search', 'sCollection', 'sCreator', 'sDelcategory', 'sFrbrtype',
       'sFulltext', 'sTitle', 'sToplevel', 'sType']

if (stage.shape[1] == production.shape[1]):
    rows = set(range(0,min(stage.shape[0],production.shape[0])))
    cols = set(range(0,stage.shape[1]))
    scolumns = stage.columns
    pcolumns = production.columns
    for row in rows:
        d = {}
        for col in cols:
            if scolumns[col] == 'sRecordid':
                if stage.iloc[row][scolumns[col]] == production.iloc[row][pcolumns[col]]:
                    d['match'] = True
                else:
                    d['match'] = False
            d[scolumns[col]] = stage.iloc[row][scolumns[col]]
            d[pcolumns[col]] = production.iloc[row][pcolumns[col]]
        comp_results = comp_results.append(d,ignore_index=True)


